# 核密度分析
## 导入相关库

In [1]:
import os
import json
import numpy as np
import geopandas as gpd
from scipy.stats import gaussian_kde
import rasterio
from rasterio.transform import from_origin
from rasterio.mask import mask
from rasterio.warp import calculate_default_transform, reproject, Resampling
from rasterio.crs import CRS

## 定义核密度分析函数

In [ ]:
def generate_kde(city, boundary_path, points_path, out_dir="output", n_pixels=1000, sigma=1000):
    os.makedirs(out_dir, exist_ok=True)
    print(f"🔄 Processing {city}...")

    # 1. 读取数据并投影为 EPSG:3857
    boundary = gpd.read_file(boundary_path).to_crs(epsg=3857)
    points = gpd.read_file(points_path).to_crs(epsg=3857)
    crs_3857 = boundary.crs

    # 2. KDE 计算
    x = points.geometry.x
    y = points.geometry.y
    xy = np.vstack([x, y])
    kde = gaussian_kde(xy, bw_method=sigma / max(xy.std(axis=1)))

    # 3. 创建网格
    xmin, ymin, xmax, ymax = boundary.total_bounds
    xres = (xmax - xmin) / n_pixels
    yres = (ymax - ymin) / n_pixels
    xx, yy = np.mgrid[xmin:xmax:n_pixels*1j, ymin:ymax:n_pixels*1j]
    positions = np.vstack([xx.ravel(), yy.ravel()])
    z = np.reshape(kde(positions).T, xx.shape)
    transform = from_origin(xmin, ymax, xres, yres)

    # 4. 保存为临时 GeoTIFF（未裁剪）
    temp_path = os.path.join(out_dir, f"{city}_kde_3857.tif")
    with rasterio.open(
        temp_path, "w",
        driver="GTiff",
        height=z.shape[0],
        width=z.shape[1],
        count=1,
        dtype=z.dtype,
        crs=crs_3857,
        transform=transform,
    ) as dst:
        dst.write(z, 1)

    # 5. 裁剪到边界
    geometry = [json.loads(boundary.to_json())["features"][0]["geometry"]]
    with rasterio.open(temp_path) as src:
        out_image, out_transform = mask(src, geometry, crop=True)
        out_meta = src.meta.copy()
        out_meta.update({
            "height": out_image.shape[1],
            "width": out_image.shape[2],
            "transform": out_transform
        })
        clipped_path = os.path.join(out_dir, f"{city}_kde_3857_clipped.tif")
        with rasterio.open(clipped_path, "w", **out_meta) as dest:
            dest.write(out_image)
    print(f"✅ Saved {clipped_path}")

    # 6. 重投影为 EPSG:4326
    dst_crs = CRS.from_epsg(4326)
    with rasterio.open(clipped_path) as src:
        transform, width, height = calculate_default_transform(
            src.crs, dst_crs, src.width, src.height, *src.bounds
        )
        meta = src.meta.copy()
        meta.update({
            "crs": dst_crs,
            "transform": transform,
            "width": width,
            "height": height
        })
        reprojected_path = os.path.join(out_dir, f"{city}_kde_4326_clipped.tif")
        with rasterio.open(reprojected_path, "w", **meta) as dst:
            reproject(
                source=rasterio.band(src, 1),
                destination=rasterio.band(dst, 1),
                src_transform=src.transform,
                src_crs=src.crs,
                dst_transform=transform,
                dst_crs=dst_crs,
                resampling=Resampling.bilinear
            )
    print(f"🌍 Reprojected and saved: {reprojected_path}")

In [ ]:
def batch_kde():
    city_list = ["tokyo", "melbourne", "rome"]
    base = "urban_data_final"
    for city in city_list:
        generate_kde(
            city=city,
            boundary_path=f"{base}/{city}_boundary.geojson",
            points_path=f"{base}/{city}_cafes.geojson"
        )

if __name__ == "__main__":
    batch_kde()